# PyrVision - Model de detecció automàtica de mamifers del Pirineu Català.

In [8]:
import csv
from ultralytics import YOLO
from pathlib import Path
from PIL import Image
from datetime import datetime

#### Opcions de PyrVision

In [9]:
#Valor de confiança en les prediccions en una escala de 0-1
conf_COLOR = 0.60
conf_BW = 0.60

#Altres opcions
rectangle=True

#Ruta de la carpeta a vigilar
watchDirectory = Path("inbox")

#Ruta de les prediccions
prediction_labels_path = Path("runs","detect","predict","labels")
prediction_path = Path("runs","detect","predict")

Per coneixer quines espècies el model és capaç de detectar, carreguem el fitxer de classes del model.

In [10]:
# Obrir el fitxer CSV
with open(Path("models","especies.csv"), 'r') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Initialize an empty list
    especies = []

    # Iterate over each row in the CSV file
    for row in reader:
        # Append the row values to the list
        especies.append(row)

# Print the list of data
for especie in especies:
    print(especie)

['Bos_taurus']
['Capreolus_capreolus']
['Felis_silvestris']
['Lepus_europaeus']
['Rupicapra_pyrenaica']
['Sus_scrofa']
['Vulpes_vulpes']
['Meles_meles']
['Cervus_elaphus']
['Sciurus_vulgaris']
['Ursus_arctos']
['Homo_sapiens']
['Dama_dama']
['Canis_familiaris']
['Equus_ferus']
['Canis_lupus']
['Vehicle']
['Martes_sp']
['Ovis_aries']
['Capra_aegagrus']


### Seguidament executarem una serie de funcions d'ajuda.

In [11]:
#function to return files in a directory
onlyfiles = [f.name for f in Path(watchDirectory).glob("*")]

In [12]:
def readPredictedSpecies():
    detections = []
    label=list(Path(prediction_labels_path).glob("*"))
    if len(label) != 0: 
        with open(Path(label[0]), 'r') as file:
        # Read classes
            linies = file.readlines()
            for linia in linies:
        # Remove leading/trailing whitespace and newline characters
                especie = linia.split()
                detections.append(especie[0])

        # Print the first value
        return (detections)
    else:
        return

In [13]:
def storefiles(file):
    especie = readPredictedSpecies()
    if (especie is None) !=0:
        #Si no hi ha cap detecció a la imatge
        Path("predicted","nuls").mkdir(parents=True, exist_ok=True)
        Path(watchDirectory,file).rename(Path("predicted",file))
        prediction_labels_path.rmdir()
        [i.rename(Path("predicted","nuls",i.name)) for i in Path(prediction_path).glob("*")]
    else:
        Path("predicted", *especies[int(especie[0])]).mkdir(parents=True, exist_ok=True)
        Path(watchDirectory,file).rename(Path("predicted",file))
        [i.rename(Path("predicted", *especies[int(especie[0])],i.name)) for i in Path(prediction_labels_path).glob("*")]
        prediction_labels_path.rmdir()
        [i.rename(Path("predicted", *especies[int(especie[0])],i.name)) for i in Path(prediction_path).glob("*")]
    prediction_path.rmdir()

In [14]:
for file in onlyfiles:
    im = Image.open(Path(watchDirectory, file))
    w,h = im.size
    impalette = im.getpixel((w/2,h/2))
    if impalette[0] == impalette [1] == impalette [2]:
        model = YOLO(Path("models","best_BW.pt"))
        imatge = Path(watchDirectory, file)
        results = model.predict(imatge, save_txt=True, save=True, show_boxes=rectangle, conf=conf_BW)
    else:
        model = YOLO(Path("models","best_COLOR.pt"))
        imatge = Path(watchDirectory, file)
        results = model.predict(imatge, save_txt=True, save=True, show_boxes=rectangle, conf=conf_COLOR)
    readPredictedSpecies
    storefiles(file)


image 1/1 /Users/arnau/classification/inbox/IMAG1875.JPG: 672x896 1 Dama_dama, 4582.5ms
Speed: 9.2ms preprocess, 4582.5ms inference, 2.1ms postprocess per image at shape (1, 3, 672, 896)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels

image 1/1 /Users/arnau/classification/inbox/IMAG0009.JPG: 672x896 2 Capreolus_capreoluss, 4673.9ms
Speed: 10.0ms preprocess, 4673.9ms inference, 2.5ms postprocess per image at shape (1, 3, 672, 896)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels

image 1/1 /Users/arnau/classification/inbox/IMAG0091.JPG: 672x896 1 Capreolus_capreolus, 1 Dama_dama, 4617.2ms
Speed: 9.1ms preprocess, 4617.2ms inference, 2.3ms postprocess per image at shape (1, 3, 672, 896)
Results saved to runs/detect/predict
1 label saved to runs/detect/predict/labels
